In [1]:
import numpy as np
import pandas as pd
from imt_lightcurve.models.lightcurve import LightCurve

Loading BokehJS ...

In [56]:
LIGHTCURVE = '105833549'
curve_id = int(LIGHTCURVE)

# Importing lightcurve data from github
data = pd.read_csv('https://raw.githubusercontent.com/Guilherme-SSB/IC-CoRoT_Kepler/main/resampled_files/' + LIGHTCURVE + '.csv')
time = data.DATE.to_numpy()
flux = data.WHITEFLUX.to_numpy()

# Create the LightCurve object
curve = LightCurve(time=time, flux=flux)
curve.plot()

In [57]:
folded_curve = curve.fold(str(curve_id))
folded_curve.plot()

In [58]:
## Orbital period values to be considered
period_real = LightCurve.get_true_value(curve_id, 'Per'); print(f'period = {period_real}')

## Radius values of the planet compared to the star
p_real = LightCurve.get_true_value(curve_id, 'Rp/R*'); print(f'p = {p_real}')

## Orbital radius values compared to star radius
adivR_real = LightCurve.get_true_value(curve_id, 'a/R*'); print(f'adivR = {adivR_real}')

## Transit impact parameter
b_real = LightCurve.get_true_value(curve_id, 'b'); print(f'b = {b_real}')


period = 2.994279
p = 0.1039
adivR = 6.71
b = 0.83


In [59]:
# Defining grid of parameters to search
period_values = LightCurve.define_interval_period(period_real); print(period_values)
p_values = LightCurve.define_interval_p(p_real); print(p_values)
adivR_values = LightCurve.define_interval_adivR(adivR_real); print(adivR_values)
b_values = LightCurve.define_interval_b(b_real); print(b_values)

[2.9941 2.9942 2.9943 2.9944 2.9945 2.9946]
[0.1037 0.1038 0.1039 0.104  0.1041 0.1042]
[6.69 6.7  6.71 6.72 6.73]
[0.81 0.82 0.83 0.84 0.85 0.86]


In [5]:
final_table = pd.read_csv('FINAL_TABLE.csv')
final_table.head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei
0,100725706.0,13.240160,13.2404,0.000141,0.1483,0.1481,0.000171,22.36,22.38,0.017078,0.92,0.95,0.000000e+00,24.875320,butterworth,n1,f01,NaN
1,101086161.0,6.212380,6.2127,0.000171,0.0892,0.0890,0.000141,11.93,11.94,0.014142,0.82,0.85,2.220446e-16,36.769832,butterworth,n1,f01,NaN
2,101368192.0,4.256670,4.2565,0.000171,0.0661,0.0659,0.000050,8.23,8.21,0.014142,0.48,0.51,0.000000e+00,93.144779,butterworth,n1,f01,NaN
3,102671819.0,2.828042,2.8283,0.000171,0.1309,0.1307,0.000112,7.81,7.82,0.014142,0.63,0.66,0.000000e+00,74.338541,butterworth,n1,f01,NaN
4,102764809.0,4.037928,4.0382,0.000171,0.1116,0.1114,0.000112,9.13,9.14,0.014142,0.74,0.77,2.220446e-16,49.546542,butterworth,n1,f01,NaN


In [7]:
final_table['avg_error'] = (final_table['e_period'] + final_table['e_p'] + final_table['e_adivR'] + final_table['e_b']) / 4

In [15]:
final_table.sort_values(by = ['chi2', 'avg_error']).head()

,CoRoT_ID,period_deleuil,period,e_period,p_deleuil,p,e_p,adivR_deleuil,adivR,e_adivR,b_deleuil,b,e_b,chi2,filter_technique,filter_order,filter_cutoff,filter_numNei,avg_error
739,101086161.0,6.212380,6.2124,0.000171,0.0892,0.0890,0.000171,11.93,11.95,0.014142,0.82,0.81,0.017078,0.899061,butterworth,n5,f06,NaN,0.007890
901,101086161.0,6.212380,6.2124,0.000171,0.0892,0.0890,0.000171,11.93,11.95,0.014142,0.82,0.81,0.017078,0.905785,butterworth,n6,f06,NaN,0.007890
1360,105833549.0,2.994279,2.9946,0.000171,0.1039,0.1037,0.000171,6.71,6.73,0.014142,0.83,0.86,0.014142,0.909329,median,NaN,NaN,7.0,0.007156
577,101086161.0,6.212380,6.2122,0.000171,0.0892,0.0890,0.000171,11.93,11.91,0.014142,0.82,0.81,0.017078,0.914735,butterworth,n4,f06,NaN,0.007890
415,101086161.0,6.212380,6.2126,0.000171,0.0892,0.0890,0.000171,11.93,11.92,0.014142,0.82,0.81,0.017078,0.953360,butterworth,n3,f06,NaN,0.007890


In [62]:
# filtered_curve = curve.butterworth_lowpass_filter(6, 0.6)
filtered_curve = curve.median_filter(7)
filtered_curve.view_filtering_results()
aux = LightCurve(filtered_curve.time, filtered_curve.filtered_flux)
folded_filtered_curve = aux.fold(str(curve_id))

In [64]:
help(multi_line_plot)

Help on function multi_line_plot in module imt_lightcurve.visualization.data_viz:

multi_line_plot(x_data=None, y1_data=None, y2_data=None, label_y1='y1 Data', label_y2='y2 Data', title='Multi-Line Plot', x_axis='x-axis', y_axis='y-axis', x_range=None, y_range=None, x_axis_type='auto', y_axis_type='auto')



In [69]:
from imt_lightcurve.visualization.data_viz import multi_line_plot

multi_line_plot(
    folded_curve.time, 
    folded_curve.flux,
    folded_filtered_curve.flux,
    label_y1='Raw Folded LC',
    label_y2='Filtered Folded LC',
    title='Folded LC x Filtered LC',
    y_axis='Normalized flux'
    )